# Boundary conditions

    For our first model, we have chosen the following conditions to simplify as much as possible :
        - The buildings exchanges with the outdoor, the outdoor temperature is homogenous
        - The radiations of the sun are diffuse, we have the same flux everywhere on the walls
        - We consider that the heat exchanges with the ground are negligible, we suppose that the floor is perfectly insulated
        - We consider that the roof top has the same thermical properties as the walls
        - We consider that the indoor temperature is homogenous
        - We neglect long wave radiation
        
    The final flux that we have for this model are : 
        - Sun's radiaitons that are homogenous all around the building, including the roof 
        - Conductive and convective exchanges with the outdoor

In [69]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dm4bem

The wall is composed of two materials: concrete and insulation. The wall 1 in normally insolated and wall 2 is supposed to be perfectly insolated. The windows are simple glazed glass windows.  The door is in PVC. The thermal properties of the materials of the wall are:

Let's consider the surface area of the walls 6 x 3 m² & 10 x 3 m² and the volume of the indoor air is 6 x 10 x 3 m³.

In [70]:
V_air = 6 * 10 * 3   # m³, indoor air volume

In [71]:
air = {'Density': 1.2,                      # kg/m³
       'Specific heat': 1000}               # J/(kg·K)
# pd.DataFrame.from_dict(air, orient='index', columns=['air'])
pd.DataFrame(air, index=['Air'])

,Density,Specific heat
Air,1.2,1000


In [72]:
concrete = {'Conductivity': 1.400,
            'Density': 2300.0,
            'Specific heat': 880,
            'Width': 0.2,
            'Surface': 5*3-1*3+10*3-1.2*3+5*3-1*3+10*3}

insulation = {'Conductivity': 0.040,
              'Density': 16.0,
              'Specific heat': 1210,
              'Width': 0.1,
              'Surface': 5*3-1*3+10*3-1.2*3+5*3-1*3+10*3}



glass = {'Conductivity': 1.4,
         'Density': 2500,
         'Specific heat': 1210,
         'Width': 0.04,
         'Surface': 2*3}
         
pvc = {'Conductivity': 0.2,
         'Density': 1.4,
         'Specific heat': 840,
         'Width': 0.05,
         'Surface': 1.2*3}
         

wall = pd.DataFrame.from_dict({'Layer_out': insulation,
                               'Layer_in': concrete,
                               'Glass': glass,
                               'Pvc': pvc },
                              orient='index')
wall

,Conductivity,Density,Specific heat,Width,Surface
Layer_out,0.04,16.0,1210,0.10,80.4
Layer_in,1.40,2300.0,880,0.20,80.4
Glass,1.40,2500.0,1210,0.04,6.0
Pvc,0.20,1.4,840,0.05,3.6


In [73]:
air = {'Density': 1.2,
       'Specific heat': 1000}

pd.DataFrame(air, index=['Air'])

,Density,Specific heat
Air,1.2,1000


In [74]:
# convection coefficients, W/(m²·K)
h = pd.DataFrame([{'in': 4+4., 'out': 10.}], index=['h'])
h

,in,out
h,8.0,10.0


# Let us start to draw the thermal circuit
## Thermal circuit

Heat transfert is:
- through the walls (concrete and insulation), 
- through the double glazed glass windows,

The HVAC system is modelled as a proportional controller. There is long wave radiative exchange between the wall and the glass window. The sources are:
- temperature sources:
    - outdoor atmospheric air;
    - indoor air temperature setpoint;
- flow rate sources:
    - solar radiation on the outdoor and the indoor walls;
    - auxiliary heat gains in the thermal zone.


### Let us consider the thermal conductances for conduction, convection, long-wave radiation, and advection.

#### Thermal conductance
$$G_{cd} = \frac{\lambda}{w}S$$


In [75]:
# conduction
G_cd = wall['Conductivity'] / wall['Width'] * wall['Surface']
pd.DataFrame(G_cd, columns={'Conductance'})

,Conductance
Layer_out,32.16
Layer_in,562.80
Glass,210.00
Pvc,14.40


#### Convection
For the inside convection coefficient, we add 4 (by convention) in order to consider easily the radiating factor. 

In [76]:
# convection
Gw = h * wall['Surface'][0]     # wall
Gg = h * wall['Surface'][2]     # glass
Gd= h*wall['Surface'][3]        # door pvc


#### Advection

In [77]:
# ventilation flow rate
Va = V_air                  # m³, volume of air
ACH = 1                     # air changes per hour
Va_dot = ACH / 3600 * Va    # m³/s, air infiltration
# ventilation & advection
Gv = air['Density'] * air['Specific heat'] * Va_dot


> Table 1. Typical values for the ventilation rates (in air changes per hour, ACH) as a function of the position of windows (H. Recknagel, E. Spenger, E_R Schramek (2013), Table 1.12.1-4)

| Position of windows                     | Ventilation rate, ACH [h⁻ⁱ] |
| --------------------------------------- | ---------------------- |
| Window closed, doors closed             | 0 to 0.5 |
| Tilted window, venetian blind closed    | 0.3 to 1.5 |
| Tilted window, whitout venetian blind   | 0.8 to 4.0 |
| Window half opened                      | 5 to 10 |
| Window fully open                       | 9 to 15 |
| Window and French window fully open (cross ventilation) | about 40 |


In [78]:
# glass: convection outdoor & conduction
Ggs = float(1 / (1 / Gg['out'] + 1 / (2 * G_cd['Glass'])))

#### Resistances

We have

$$R_{cd} = \frac{w}{\lambda S}$$

where:

- $w$ is the [width](https://en.m.wikipedia.org/wiki/Length) of the material, m;
- $\lambda$ - [thermal conductvity](https://en.m.wikipedia.org/wiki/Thermal_conductivity) of the material, W/(m·K);
- $S$ - [surface area](https://en.m.wikipedia.org/wiki/Surface_area) of the wall, m².

In [91]:
he=h["out"] #h ext
hi=h["in"] #h interieur
Sint2=5.4*5.6 #surface au sol piece 2
Sm1=3*5*3-1*3-1.2*3 #surface mur 1
Sm12=4.4*3 #surface mur mileu
Sp=1.2*3 # surface portes
Sf=1*3 #surface fenetres
eb=0.2 #epaisseur beton
ei=0.1 #epaisseur isolant
ef=0.01 #epaisseur fenetre
ep=0.05 # epaisseur porte
em12=0.1 #epaisseur mur mileu
lda_b =1.4 #conductivité thermique du beton
lda_i = 0.04#conductivité du beton
lda_p = 0.2#conductivité pvc
lda_f = 1.4#conductivité vitre
dens_b = 2300#densité beton
dens_i = 16#densité isolant
dens_f = 2500#densité fenêtre
dens_p = 1410#densité porte
dens_air=1.2 #densite air
cb=880 #capacité thermique beton
ci=1210
cf=1210
cp=840
cair=1000
Kp=1e9

R0=1/(he*Sm1) 
R1=R2=eb/(2*lda_b*Sm1) #accumulation de chaleur beton
R3=R4=ei/(2*lda_i*Sm1) #accumulation de chaleur isolant
R5=1/(he*Sf)+ef/(2*lda_f*Sf) #convection fenetre -ext + moitié conduction
R6=1/(he*Sp) #convectiopn ext-porte
R7=R8=ep/(2*lda_p*Sp) #accumulation
R9=1/(hi*Sm1) #convection mur -int
R10=1/(hi*Sf)+ef/(2*lda_f*Sf) #convection fenetre-int + moitié conduction
R11=1/(hi*Sp) #convection porte int
R12=Kp
R13=Kp # a changer
R14=1/(hi*Sp) #convection interieur porte
R15=1/(hi*Sm12) 
R16=R17=em12/(2*lda_p*Sp)
R18=1/(hi*Sp)
R19=R20=em12/(2*lda_b*Sm12)
R21=1/(hi*Sm12)
R22=R23=Kp
R24=1/(he*Sf)+ef/(2*lda_f*Sf)
R25=1/(hi*Sf)+ef/(2*lda_f*Sf)


#### Thermal Capacities
The [thermal capacities](https://en.m.wikipedia.org/wiki/Heat_capacity) of the wall are of the form:

$$C_w= m_w c_w= \rho_w c_w w_w S_w$$

where:
- $m_w = \rho_w w_w S_w$ is the [mass](https://en.m.wikipedia.org/wiki/Mass) of the material of the wall, kg;
- $c_w$ - [specific heat capacity](https://en.m.wikipedia.org/wiki/Specific_heat_capacity) of the material, J/(kg⋅K);
- $\rho_w$ - [density](https://en.m.wikipedia.org/wiki/Density) of the material, kg/m³;
- $w_w$ - width of the wall, m;
- $S_w$ - surface area of the wall, m²

In [80]:
C = wall['Density'] * wall['Specific heat'] * wall['Surface'] * wall['Width']

The thermal capacity of the air is:

$$C_a = m_a c_a = \rho_a c_a V_a$$

where:
- $m_a = \rho_a V_a$ is the mass of the air, kg;
- $\rho_w$ - [density](https://en.m.wikipedia.org/wiki/Density) of air, kg/m³;
- $c_a$ - specific heat capacity of the air, J/(kg⋅K);
- $V_a$ - volume of the air in the thermal zone, m³.

In [81]:
C['Air'] = air['Density'] * air['Specific heat'] * V_air
pd.DataFrame(C, columns={'Capacity'})

,Capacity
Layer_out,155654.40
Layer_in,32545920.00
Glass,726000.00
Pvc,211.68
Air,216000.00


## Differential algebraic equations (DAE)

In [82]:
# number of temperature nodes and flow branches
no_θ = 18 #nodes
no_q = 26 #flow

### Conductance matrix $G$

In [83]:
G=np.eye(26)

G[0][0]=1/R0
G[1][1]=1/R1
G[2][2]=1/R2
G[3][3]=1/R3
G[4][4]=1/R4
G[5][5]=1/R5
G[6][6]=1/R6
G[7][7]=1/R7
G[8][8]=1/R8
G[9][9]=1/R9
G[10][10]=1/R10
G[11][11]=1/R11
G[12][12]=1/R12
G[13][13]=1/R13
G[14][14]=1/R14
G[15][15]=1/R15
G[16][16]=1/R16
G[17][17]=1/R17
G[18][18]=1/R18
G[19][19]=1/R19
G[20][20]=1/R20
G[21][21]=1/R21
G[22][22]=1/R22
G[23][23]=1/R23
G[24][24]=1/R24
G[25][25]=1/R25



### Capacity matrix $C$

In [93]:
C=np.zeros(18)

C[1] = dens_b*eb*Sm1*cb
C[3] = dens_i*ei*Sm1*ci
C[5] = dens_f*ef*Sf*cf
C[7] = dens_p*ep*Sp*cp
C[9] = dens_air*Sint2*3*cair
C[11] = dens_p*Sp*ep*cp
C[14] = dens_b*Sm12*em12*cb
C[16] = dens_air*Sint2*3*cair
C[17] = Sf*ef*cf*dens_f
C=np.diag(C)



### Arc-node incidence matrix $A$

In [85]:
# Arc-node incidence matrix
A = np.zeros((no_q, no_θ ))
for i in range (10):
    A[i][i]=1
A[4][3]=-1
A[9][4]=-1
A[1][0]=-1
A[2][1]=-1
A[3][2]=-1
A[7][6]=-1
A[8][7]=-1
A[11][8]=-1
A[25][17]=-1
A[24][17]=1
A[23][16]=1
A[21][16]=1
A[20][15]=1
A[19][14]=1
A[18][16]=1
A[17][12]=1
A[16][11]=1
A[15][13]=-1
A[14][10]=-1
A[22][16]=1
A[21][15]=-1
A[20][14]=-1
A[19][13]=-1
A[18][12]=-1
A[17][11]=-1
A[16][10]=-1
A[15][9]=1
A[14][9]=1
A[13][9]=1
A[11][8]=-1
A[10][5]=-1
A[9][4]=-1
A[12][9]=1
A[11][9]=1
A[10][9]=1
pd.DataFrame(A)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### We are now defining the temperature vector $b$ and the heat flow vector $f$.
We first define the temperature vector. We have two different temperatures. The temperature of the outise $T_{out}=15°C$ and the wanted temperature $T_{HVAC}=20°C$

In [86]:
Thvac=20
Tout=15
b=np.zeros((26,1))

b[0,0]=1
b[5,0]=1
b[6,0]=1
b[12,0]=1
b[13,0]=1
b[22,0]=1
b[23,0]=1
b[24,0]=1

Now, for the flux, we have two different flux. We have the flux of the sun that radiate on all the outside surfaces $\Phi_{sun_{out}}$ and the flux of the sun that radiat eon the insides surfaces $\Phi_{sun_{in}}$. Finally there are two people bringing $100W$ each in the fist room and a $40W$ lamp in each room. 
The flux in the first room is $$\dot Q_{a1}=240W$$
And the flux in the second room $$\dot Q_{a2}=40W$$




In [125]:
Qa1 = 240
Qa2 = 40
f = np.zeros(18)

f[0] = 1
f[4] = 1
f[5] = 1
f[9] = 1
f[16] = 1


We have to define the y vector that will help us to solvve the equations.

In [89]:
y = np.zeros(18)         # nodes
y[[9,16]] = 1             # nodes (temperatures) of interest
print(f'y = ', y)

y =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0.]


We want to obtain the space representation of the model. To do that we transforme the differential-algebraic system of equations : $$C \dot{\theta} = -(A^T G A) \theta + A^T G b + f$$
in $$\left\{\begin{array}{rr}
\dot{\theta}_s=A_s \theta_C + B_s u\\ 
y = C_s \theta_s + D_s u
\end{array}\right.$$We have to comput As, Bs, Cs and Ds that represent the state coefficient.

In [126]:
[As, Bs, Cs, Ds] = dm4bem.tc2ss(A, G, b, C, f, y)
print('As = \n', As, '\n')
print('Bs = \n', Bs, '\n')
print('Cs = \n', Cs, '\n')
print('Ds = \n', Ds, '\n')


As = 
 [[-1.62798668e-05  1.86945839e-06  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 3.90886755e-04 -7.66544155e-04  0.00000000e+00  0.00000000e+00
   3.75657400e-04  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -5.76295874e-04  0.00000000e+00
   2.57116621e-04  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.42594469e-04
   6.75447484e-05  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  2.56533590e-04  2.14334705e-04  1.32275132e-04
  -1.44578501e-03  8.81834215e-05  7.54458162e-04  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   4.50298323e-05 -9.00596645e-05  0.00000000e+00  4.50298323e-05
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   3.07422047e-05  0.

Now, we have to verify if the state representation and so our model isn't false. To do that, we study the steady state. At steady state the term $C \dot{\theta} = 0$. Moreover, the flow rate sources are equal to zero, outdoor temperature is 15°C and indoor temperature setpoint (temperature of the HVAC) is 20°C. We are going to execute two tests. one with the HVAC system on ($K_{p}\ne0$). And one with the HVAC off ($K_{p}=0$)

In [107]:
f = np.zeros(18) #for steady state
b=np.zeros(26)
b[[0,5,6,24,12,22]]=Tout
b[[13,23]]=Thvac


The objective is to check if with the system of DAE we obtain approximatively the same indoor temperature as the state-space representation. 
### Test 1, HVAC system on ($K_{p}\ne0$)
- First step : value of temperature in steady state with the system of DAE by considering that $C\dot{\theta} = 0$. 
We obtain $$\theta_{ss} = (A^T G A)^{-1}(A^T G b + f)$$

In [108]:
θ = np.linalg.inv(A.T @ G @ A) @ (A.T @ G @ b + f)
print(f'θ = {θ} °C')

θ = [15.         15.         15.         15.         15.         15.
 15.         15.         15.         15.         15.         15.
 15.         15.         15.         15.         15.          8.30769231] °C


We obtain a vector with all the temperature of the model. Normally we must obtain an indoor temperature equal to 20°C because the HVAC si ON.

- Second step, value of temperature in steady state with state-space representation
To resolve the system we have to input the vector u. It is obtained by stacking the vectors $b_{T}$ and $f_{Q}$. $b_{T}$ and $f_{Q}$ respectively correspond to the temperature values of the model and to flow values of the model.
$b_{T}$ and $f_{Q}$ are non zero elements of vector $b$ and $f$


In [113]:
bT=np.ones(8)
bT*=Tout
bT[4],bT[6]=Thvac,Thvac

fQ=np.ones(5)
phi_sun = 400 #W/m2
fQ[0,0] = phi_sun*Sm1*0.6
fQ[4,0] = phi_sun*Sf*0.15
fQ[5,0] = phi_sun*Sp*0.6
fQ[9,0] = Qa1
fQ[16,0] = Qa2



[15. 15. 15. 15. 20. 15. 20. 15.  1.  1.  1.]


As we are in steady state, the flow rate sources are taken equal to 0.

In [116]:
fQ=np.zeros(5)
u = np.hstack([bT, fQ])
print (u)

[15. 15. 15. 15. 20. 15. 20. 15.  0.  0.  0.  0.  0.]


Then, at steady-state $C\dot{\theta} = 0$. We obtain the indoor temperature thanks to $$y_{ss} = (-C_s A_s^{-1} B_s + D_s) u$$

In [127]:
yss = (-Cs @ np.linalg.inv(As) @ Bs + Ds) @ u
print(f'yss = {yss} °C')

yss = [15. 15.] °C


### Test 2, HVAC system off ($K_{p}=0$)
they must be equal to 15°C if the regulator is equal 0 ($K_{p} = 0$) because that mean that the HVAC system is OFF.